# Predicción de Riesgo de Enfermedad Cardíaca: Tarea de Regresión Logística

## Contexto Introductorio
Las enfermedades cardíacas son la principal causa de muerte en el mundo. Los modelos predictivos como la regresión logística pueden permitir la identificación temprana de pacientes en riesgo mediante el análisis de características clínicas. En esta tarea, implementaremos la regresión logística en el conjunto de datos de enfermedades cardíacas (muestras de pacientes con características como edad, colesterol, presión arterial).

## Instrucciones
Completar en este cuaderno Jupyter, implementando funciones de la teoría de clase (sigmoide, costo, GD). Usar NumPy, Pandas y Matplotlib.

---

## Paso 1: Cargar y Preparar el Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuración de gráficos
%matplotlib inline
plt.style.use('ggplot')

### 1.1 Cargar Datos
Cargaremos el dataset desde la carpeta `data/`.

In [ ]:
import os

data_path = 'data/'
files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
if not files:
    raise FileNotFoundError("No se encontró el archivo CSV en la carpeta data/")
csv_file = files[0]
print(f"Cargando archivo: {csv_file}")

df = pd.read_csv(os.path.join(data_path, csv_file))
df.head()

### 1.2 Preprocesamiento y EDA
Convertiremos la columna objetivo `Heart Disease` a binaria (1 para Presence, 0 para Absence) y realizaremos un análisis exploratorio básico.

In [ ]:
# Binarizar la columna objetivo
df['Heart Disease'] = df['Heart Disease'].map({'Presence': 1, 'Absence': 0})

# Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# Estadísticas descriptivas
df.describe()

In [ ]:
# Distribución de clases
plt.figure(figsize=(6, 4))
df['Heart Disease'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Distribución de la Enfermedad Cardíaca (0=Ausencia, 1=Presencia)')
plt.xlabel('Clase')
plt.ylabel('Cantidad')
plt.xticks(ticks=[0, 1], labels=['Ausencia', 'Presencia'], rotation=0)
plt.show()

### 1.3 Selección de Características y Normalización
Utilizaremos una selección de características indicadas y normalizaremos los datos.

In [ ]:
# Selección de características (features) y objetivo (target)
# Seleccionamos >= 6 características como se solicita
features = ['Age', 'Cholesterol', 'BP', 'Max HR', 'ST depression', 'Number of vessels fluro']
target = 'Heart Disease'

X = df[features].values
y = df[target].values

# Normalización (Min-Max Scaling manual o usando librerías si se prefiere, aquí manual para ejercicio)
def normalize(X):
    min_val = np.min(X, axis=0)
    max_val = np.max(X, axis=0)
    return (X - min_val) / (max_val - min_val), min_val, max_val

X_norm, min_val, max_val = normalize(X)

print("Forma de X:", X_norm.shape)
print("Forma de y:", y.shape)

### 1.4 División Train/Test
Dividiremos los datos en 70% entrenamiento y 30% prueba de manera estratificada.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, stratify=y, random_state=42)

print(f"Train set: {X_train.shape[0]} muestras")
print(f"Test set: {X_test.shape[0]} muestras")

---

## Paso 2: Implementar Regresión Logística Básica
Implementaremos las funciones `sigmoid`, `cost` y `gradient_descent` desde cero.

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z
    """
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w, b):
    """
    Calcula la función de costo (Binary Cross-Entropy)
    """
    m = X.shape[0]
    z = np.dot(X, w) + b
    f_wb = sigmoid(z)
    
    # Evitar log(0) añadiendo un epsilon pequeño
    epsilon = 1e-15
    cost = -1/m * np.sum(y * np.log(f_wb + epsilon) + (1 - y) * np.log(1 - f_wb + epsilon))
    return cost

def compute_gradient(X, y, w, b):
    """
    Calcula el gradiente para el descenso de gradiente
    """
    m, n = X.shape
    z = np.dot(X, w) + b
    f_wb = sigmoid(z)
    
    err = f_wb - y
    dj_dw = (1/m) * np.dot(X.top, err) if hasattr(X, 'top') else (1/m) * np.dot(X.T, err)
    dj_db = (1/m) * np.sum(err)
    
    return dj_dw, dj_db

def gradient_descent(X, y, w_in, b_in, alpha, num_iters):
    """
    Realiza el descenso de gradiente para aprender w y b
    """
    m = X.shape[0]
    J_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    
    for i in range(num_iters):
        dj_dw, dj_db = compute_gradient(X, y, w, b)
        
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        if i % 100 == 0:
            cost = compute_cost(X, y, w, b)
            J_history.append(cost)
            # print(f"Iteración {i:4d}: Costo {cost:8.2f}") # Opcional: imprimir progreso
            
    return w, b, J_history

### 2.1 Entrenamiento del Modelo
Entrenaremos el modelo utilizando el conjunto de entrenamiento completo.

In [ ]:
import copy

# Inicialización de parámetros
m, n = X_train.shape
initial_w = np.zeros(n)
initial_b = 0.
iterations = 2000
alpha = 0.01

print("Iniciando entrenamiento...")
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b, alpha, iterations)
print(f"Entrenamiento completado. w: {w_final}, b: {b_final:0.2f}")

### 2.2 Gráfica de Costo vs Iteraciones

In [ ]:
plt.plot(J_hist)
plt.title("Costo vs Iteraciones")
plt.ylabel('Costo')
plt.xlabel('Iteraciones (x100)')
plt.show()

### 2.3 Evaluación del Modelo
Evaluaremos el modelo en los conjuntos de entrenamiento y prueba.

In [ ]:
def predict(X, w, b, threshold=0.5):
    z = np.dot(X, w) + b
    p = sigmoid(z)
    return (p >= threshold).astype(int)

def evaluate_metrics(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    accuracy = (tp + tn) / len(y_true)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return accuracy, precision, recall, f1

# Predicciones
y_train_pred = predict(X_train, w_final, b_final)
y_test_pred = predict(X_test, w_final, b_final)

# Métricas
metrics_train = evaluate_metrics(y_train, y_train_pred)
metrics_test = evaluate_metrics(y_test, y_test_pred)

results_df = pd.DataFrame([metrics_train, metrics_test], 
                          columns=['Accuracy', 'Precision', 'Recall', 'F1 Score'], 
                          index=['Train', 'Test'])
results_df